In [1]:
import pandas as pd
import re
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, classification_report, log_loss
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix


# Database connection setup
db_config = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123',
    'port': '5432'
}
connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

# Load data from PostgreSQL
query = 'SELECT * FROM "corrected_merged_claim_data_EF";'
df = pd.read_sql(query, con=engine)

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

df_filtered = df[df['Policy Status'].isin(['Renewed', 'Not Renewed'])]

df_filtered['Status_Binary'] = df_filtered['Policy Status'].apply(lambda x: 1 if x == 'Not Renewed' else 0)

# Select relevant columns
selected_columns = ['Renewal Type', 'Product name ', 'Product name  2', 'biztype', 'age', 'Vehicle IDV', 
                    'Before GST Add-on GWP', 'Total OD Premium', 'Total TP Premium', 'gst', 'Total Premium Payable ', 'NCB % Previous Year', 
                    'Applicable Discount with NCB', 'Tie Up', 'Number of claims', 'Approved', 'Denied', 
                    'Status_Binary', 'Policy Tenure', 'Customer Tenure', 'New Customers', 'Claim Happaned/Not', 'Renewal Rate Status', 'WITHDRAWN']
df_selected = df_filtered[selected_columns]

# Convert numerical columns to float
numerical_columns = [ 'age', 'Vehicle IDV', 
                    'Before GST Add-on GWP', 'Total OD Premium', 'Total TP Premium', 'gst', 'Total Premium Payable ', 'NCB % Previous Year', 
                    'Applicable Discount with NCB', 'Number of claims', 'Approved', 'Denied', 
                    'Status_Binary', 'Policy Tenure', 'Customer Tenure', 'WITHDRAWN']
df_selected[numerical_columns] = df_selected[numerical_columns].apply(pd.to_numeric, errors='coerce')

# One-hot encode categorical columns
categorical_columns = ['Renewal Type', 'Product name ', 'Product name  2', 'biztype', 'Tie Up',
                       'New Customers', 'Claim Happaned/Not', 'Renewal Rate Status']
df_selected = pd.get_dummies(df_selected, columns=categorical_columns, drop_first=True)

# Impute missing values
imputer = SimpleImputer(strategy='constant', fill_value=0)
df_selected = pd.DataFrame(imputer.fit_transform(df_selected), columns=df_selected.columns)

# Convert to sparse matrix
X = csr_matrix(df_selected.drop(columns=['Status_Binary']).values)
y = df_selected['Status_Binary']

# Debugging dimensions
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

C:\Users\Admin\AppData\Local\Temp\ipykernel_1452\3154240862.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Status_Binary'] = df_filtered['Policy Status'].apply(lambda x: 1 if x == 'Not Renewed' else 0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_1452\3154240862.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected[numerical_columns] = df_selected[numerical_columns].apply(pd.to_numeric, errors='coerce')


Shape of X: (1280454, 47)
Shape of y: (1280454,)


In [3]:
import numpy as np
import pandas as pd
import xgboost as xgb
import re
from collections import defaultdict

# feature dependencies
all_feature_dependencies = defaultdict(list)

# Prepare feature names
feature_names = df_selected.drop(columns=['Status_Binary']).columns.tolist()

# Train XGBoost model
print("\nTraining XGBoost Model - Single Run")

xgb_model = xgb.XGBClassifier(
    max_depth=5,                  
    learning_rate=0.05,            
    n_estimators=200,              
    subsample=0.8,                 
    colsample_bytree=0.8,         
    scale_pos_weight=len(y[y == 0]) / len(y[y == 1]),  
    gamma=0.1,                     
    random_state=42  
) 

xgb_model.fit(X, y)

# Extract tree dump (all trees)
booster = xgb_model.get_booster()
tree_dumps = booster.get_dump()

formatted_trees = []

# Process each tree separately
for tree_index, tree_dump in enumerate(tree_dumps):
    formatted_tree = []
    indent = "    "  

    for line in tree_dump.split("\n"):
        if line.strip():  
            depth = line.count("\t")  
            formatted_line = line.replace("\t", indent)  

            # Replace feature indices (f0, f1, ...) with actual feature names
            for i, feature in enumerate(feature_names):
                formatted_line = formatted_line.replace(f"f{i}<", f"{feature} < ")
                formatted_line = formatted_line.replace(f"f{i}>", f"{feature} > ")

            # Fix incorrect thresholds (< 2.00001001 → < 0.5)
            formatted_line = re.sub(r"<\s*2\.00001001", "< 0.5", formatted_line)
            formatted_line = re.sub(r">=\s*2\.00001001", ">= 0.5", formatted_line)

            # Convert leaf values into class labels
            match = re.search(r"leaf=([-+]?\d*\.\d+)", formatted_line)
            if match:
                logit = float(match.group(1))
                probability = 1 / (1 + np.exp(-logit))  
                predicted_class = 1 if probability > 0.5 else 0  
                formatted_line = re.sub(r"leaf=[-+]?\d*\.\d+", f"class: {predicted_class}", formatted_line)

            formatted_line = formatted_line.replace("[", "|--- ").replace("]", "")
            formatted_tree.append(indent * depth + formatted_line)

    formatted_trees.append("\n".join(formatted_tree))

# Save formatted trees
with open("xgboost_tree_run.txt", "w") as file:
    file.write("\n\n".join(formatted_trees))

print("Decision trees saved as 'xgboost_tree_run_balanceddata.txt'")



Training XGBoost Model - Single Run
Decision trees saved as 'xgboost_tree_run_balanceddata.txt'


In [4]:
import re
from collections import defaultdict

# Load the tree rules from the file
file_path = "xgboost_tree_run.txt"

with open(file_path, "r") as file:
    tree_rules = file.readlines()

# Automatically get feature names from dataset
df_selected_columns = [
    "Renewal Rate Status_Null", "age", "gst", "Tie Up_HYUNDAI", "Before GST Add-on GWP", "Vehicle IDV",
    "Total OD Premium", "biztype_Roll Over", "Policy Tenure", "Applicable Discount with NCB", "Tie Up_MARUTI",
    "Total TP Premium", "Approved", "Renewal Type_TY Onwards", "Renewal Type_SY onwards", "Product name _PrivateCarPolicy",
    "biztype_Renewal Business", "Tie Up_MIBL OEM", "Tie Up_Non-OEM", "Tie Up_TATA PV",
    "Product name _Standalone Own Damage Policy for Private Car", "Product name _Private Car Policy - Bundled Cover",
    "Tie Up_EM Non OE Dealership", "New Customers_Yes", "Number of claims", "Customer Tenure", "Total Premium Payable",
    "Tie Up_FORD", "Tie Up_HONDA PV", "Tie Up_SKODA", "Tie Up_VOLKSWAGEN", "Denied", "WITHDRAWN",
    "Tie Up_EM Broker", "Claim Happened/Not_Yes", "NCB % Previous Year", "OEM", "Renewal Rate Status_Increase",
    "Renewal Type_SY", "Not_Yes", "Tie Up_MIBL Others"
]  

# Ensure exact feature name matching
def match_feature_name(extracted_name):
    for col in df_selected_columns:
        if extracted_name in col:  
            return col  
    return extracted_name  

# Dictionary to store feature dependencies and their frequency
feature_threshold_counts = defaultdict(lambda: {"<": defaultdict(int), ">=": defaultdict(int)})

# Parse tree and extract rules leading to `class: 1`
def extract_non_renewal_conditions(tree_rules):
    current_path = []  

    for line in tree_rules:
        line = line.strip()
        if not line:
            continue

        depth = line.count("|---")
        line_content = line.replace("|--- ", "").strip()

        # Handle decision nodes (conditions)
        match = re.search(r"([A-Za-z0-9_ /-]+) (<|>) ([0-9\.]+) yes=(\d+),no=(\d+)", line_content)
        if match:
            feature, operator, threshold, yes_branch, no_branch = match.groups()
            threshold = float(threshold)

            # Ensure full feature names
            feature = match_feature_name(feature)

            current_path.append((depth, feature, operator, threshold, yes_branch, no_branch))

        # Handle leaf nodes (class 1)
        elif "class: 1" in line_content:
            for depth, feature, operator, threshold, yes_branch, no_branch in current_path:
                if str(yes_branch) in line:  # `<` condition
                    feature_threshold_counts[feature]["<"][threshold] += 1
                elif str(no_branch) in line:  # `>=` condition
                    feature_threshold_counts[feature][">="][threshold] += 1

    return feature_threshold_counts

# Extract dependencies and their frequency
feature_threshold_counts = extract_non_renewal_conditions(tree_rules)

# Save reasons
output_file = "non_renewal_thresholds_fixed.txt"

with open(output_file, "w") as file:
    file.write("Key Non-Renewal Conditions (Full Feature Names Fixed):\n")
    file.write("=" * 100 + "\n\n")

    for feature, threshold_data in feature_threshold_counts.items():
        most_common_less_than = max(threshold_data["<"], key=threshold_data["<"].get, default=None)
        most_common_greater_than = max(threshold_data[">="], key=threshold_data[">="].get, default=None)

        less_than_count = threshold_data["<"].get(most_common_less_than, 0)
        greater_than_count = threshold_data[">="].get(most_common_greater_than, 0)

        file.write(f" **{feature}** influences non-renewal due to:\n")

        if most_common_less_than is not None:
            dominant_marker = "" if less_than_count > greater_than_count else ""
            file.write(f"   {dominant_marker}- {feature} < {most_common_less_than} (Count: {less_than_count})\n")

        if most_common_greater_than is not None:
            dominant_marker = "" if greater_than_count > less_than_count else ""
            file.write(f"   {dominant_marker}- {feature} >= {most_common_greater_than} (Count: {greater_than_count})\n")

        file.write("-" * 100 + "\n")

print(f"\nUpdated non-renewal thresholds saved to '{output_file}'")


Updated non-renewal thresholds saved to 'non_renewal_thresholds_fixed.txt'
